In [ ]:
import os
import time
import torch
import torch_xla.core.xla_model as xm
from transformers import AutoTokenizer, AutoModelForCausalLM

print("Étape 0: Initialisation.")
start_time = time.time()

os.environ["XLA_USE_BF16"] = "1"
os.environ["TORCH_XLA_DEBUG"] = "0"
MODEL_ID = "utter-project/EuroLLM-1.7B"

# --- ÉTAPE 1 ---
print("\nÉtape 1: Connexion au TPU...")
device = xm.xla_device()
print(f"Device TPU trouvé: {device}")

# --- ÉTAPE 2 ---
print("\nÉtape 2: Téléchargement du tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
print("Tokenizer téléchargé.")

# --- ÉTAPE 3 ---
print("\nÉtape 3: Téléchargement du modèle...")
model = AutoModelForCausalLM.from_pretrained(MODEL_ID, torch_dtype=torch.bfloat16)
model.config.use_cache = False  # XLA incompatible
model.to(device)
xm.mark_step()
print("Modèle prêt sur TPU.")

# --- ÉTAPE 4 ---
prompt = "Write a short summary of the French Revolution:"
print("\nÉtape 4: Tokenisation du prompt...")
inputs = tokenizer(prompt, return_tensors="pt").to(device)
input_ids = inputs["input_ids"]
print("Prompt tokenizé.")

# --- ÉTAPE 5 ---
print("\nÉtape 5: Warm-up TPU (compilation initiale courte)...")
with torch.no_grad():
    _ = model(input_ids)
xm.mark_step()
print("Warm-up terminé.")

# --- ÉTAPE 6 ---
print("\nÉtape 6: Génération manuelle token par token (optimisée pour TPU)...")

max_new_tokens = 64
generated_ids = input_ids
model.eval()

with torch.no_grad():
    for i in range(max_new_tokens):
        # Forward
        outputs = model(generated_ids)
        next_token_logits = outputs.logits[:, -1, :]

        # Prendre le token le plus probable (greedy)
        next_token = torch.argmax(next_token_logits, dim=-1, keepdim=True)

        # Ajouter le nouveau token à la séquence
        generated_ids = torch.cat([generated_ids, next_token], dim=-1)

        # Synchronisation TPU
        xm.mark_step()

        # Vérifie si fin de séquence atteinte
        if next_token.item() == tokenizer.eos_token_id:
            print(f"\nArrêt précoce à {i+1} tokens (EOS atteint).")
            break

        # Affiche un aperçu tous les 10 tokens
        if (i + 1) % 10 == 0:
            partial = tokenizer.decode(generated_ids[0].cpu(), skip_special_tokens=True)
            print(f"→ {i+1} tokens générés...")
            print(partial)
            print("-" * 50)

print("\nGénération terminée.")
print(f"Temps écoulé: {time.time() - start_time:.2f} secondes")

# --- ÉTAPE 7 ---
print("\nÉtape 7: Décodage final...")
generated_text = tokenizer.decode(generated_ids[0].cpu(), skip_special_tokens=True)

print("\n--- OUTPUT FINAL ---")
print(generated_text)

/usr/local/lib/python3.12/dist-packages/torch_xla/experimental/gru.py:113: SyntaxWarning: invalid escape sequence '\_'
  * **h_n**: tensor of shape :math:`(D * \text{num\_layers}, H_{out})` or


Étape 0: Initialisation du script.

Étape 1: Connexion au device TPU...
Device TPU trouvé: xla:0
Temps écoulé: 4.21 secondes

Étape 2: Téléchargement du tokenizer...


/tmp/ipython-input-4006931737.py:17: DeprecationWarning: Use torch_xla.device instead
  device = xm.xla_device()
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/960 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/2.41M [00:00<?, ?B/s]

...

Modèle prêt sur TPU.
Temps écoulé: 91.20 secondes

Étape 4: Tokenisation du prompt...
Prompt tokenizé.

Étape 5: Warm-up TPU (compilation initiale courte)...


/tmp/ipython-input-4006931737.py:48: DeprecationWarning: Use torch_xla.sync instead
  xm.mark_step()


Warm-up terminé.
Temps écoulé: 96.77 secondes

Étape 6: Génération manuelle token par token (optimisée pour TPU)...


/tmp/ipython-input-4006931737.py:72: DeprecationWarning: Use torch_xla.sync instead
  xm.mark_step()


→ 10 tokens générés...
Write a short summary of the French Revolution:
The French Revolution was a period of radical political
--------------------------------------------------
→ 20 tokens générés...
Write a short summary of the French Revolution:
The French Revolution was a period of radical political and social change in France that began in 1
--------------------------------------------------
→ 30 tokens générés...
Write a short summary of the French Revolution:
The French Revolution was a period of radical political and social change in France that began in 1789 and lasted until 179
--------------------------------------------------
→ 40 tokens générés...
Write a short summary of the French Revolution:
The French Revolution was a period of radical political and social change in France that began in 1789 and lasted until 1799. It was a time of great uphe
--------------------------------------------------
→ 50 tokens générés...
Write a short summary of the French Revolution:
The Fre